# 🏈 NFL Survivor AI - Complete Automation System

**Fully automated, production-ready NFL Survivor Pool optimization with real-time data**

---

## 📋 What This Notebook Does

This notebook provides a **100% automated workflow** for:

1. ✅ **Auto-Setup**: Clone repository, install dependencies
2. ✅ **Real-Time Data**: Fetch live odds from The Odds API
3. ✅ **Season Detection**: Auto-detect current NFL week and season
4. ✅ **ML Predictions**: Advanced machine learning models (R² > 0.75)
5. ✅ **Optimization**: Hungarian algorithm for optimal picks
6. ✅ **Risk Analysis**: Monte Carlo simulation (10,000+ iterations)
7. ✅ **Recommendations**: Top 5 picks with complete season paths

---

## 🚀 Quick Start

1. **Get API Key** (Optional but recommended):
   - Visit [The Odds API](https://the-odds-api.com/)
   - Sign up for free (500 requests/month)
   - Copy your API key

2. **Run All Cells**: Click `Runtime` → `Run all`

3. **Enter Configuration**:
   - Your API key (or skip for demo mode)
   - Pool size
   - Teams you've already used

4. **Get Results**: View optimal picks and season paths!

---

**Note**: First run takes ~3-5 minutes for setup. Subsequent runs are faster with cached data.

# 1️⃣ Automated Setup & Installation

This section automatically:
- Clones the Survivor AI repository
- Installs all required dependencies
- Sets up the environment

**⏱️ Expected time: 2-3 minutes (first run only)**

In [ ]:
#@title 🔧 Clone Repository & Setup Environment
#@markdown This cell clones the Survivor AI repository and sets up the working directory.

import os
import sys
from pathlib import Path

# Configuration
REPO_URL = "https://github.com/elliotttmiller/survivorai.git"
REPO_NAME = "survivorai"
WORK_DIR = f"/content/{REPO_NAME}"

print("🏈 NFL Survivor AI - Automated Setup")
print("=" * 60)

# Check if already cloned
if os.path.exists(WORK_DIR):
    print(f"✓ Repository already exists at {WORK_DIR}")
    print("  Updating to latest version...")
    !cd {WORK_DIR} && git pull --quiet
    print("  ✓ Updated to latest version")
else:
    print(f"📥 Cloning repository from GitHub...")
    !git clone --quiet {REPO_URL} {WORK_DIR}
    print(f"  ✓ Repository cloned to {WORK_DIR}")

# Change to working directory
os.chdir(WORK_DIR)
sys.path.insert(0, WORK_DIR)

print(f"\n📁 Working directory: {os.getcwd()}")
print("✅ Setup complete!\n")

In [ ]:
#@title 📦 Install Dependencies
#@markdown Installs all required Python packages. This may take 2-3 minutes on first run.

print("📦 Installing Python dependencies...")
print("=" * 60)
print("This may take 2-3 minutes on first run.\n")

# Install core dependencies
!pip install -q numpy pandas scipy requests beautifulsoup4 lxml python-dotenv

# Install ML dependencies
!pip install -q scikit-learn joblib

# Install XGBoost (optional, for ensemble)
!pip install -q xgboost

# Install visualization (optional)
!pip install -q matplotlib seaborn plotly

print("\n✅ All dependencies installed successfully!")
print("\n📚 Installed packages:")
print("  • Core: numpy, pandas, scipy")
print("  • Data: requests, beautifulsoup4, lxml")
print("  • ML: scikit-learn, xgboost, joblib")
print("  • Viz: matplotlib, seaborn, plotly")

In [ ]:
#@title ⚙️ Configuration & API Setup
#@markdown Configure the system with your API key and preferences.

import os
from datetime import datetime
from getpass import getpass

print("⚙️ System Configuration")
print("=" * 60)

# Prompt for API key
print("\n🔑 The Odds API Configuration")
print("   Get free key at: https://the-odds-api.com/")
print("   Free tier: 500 requests/month\n")

api_key = getpass("Enter your Odds API key (or press Enter to skip): ")

if api_key:
    os.environ['ODDS_API_KEY'] = api_key
    print("✅ API key configured (real-time data enabled)")
else:
    os.environ['ODDS_API_KEY'] = ''
    print("⚠️  No API key provided (using demo/fallback data)")
    print("   System will use SurvivorGrid and ML predictions only")

# Auto-detect season and week
print("\n📅 Auto-detecting NFL season...")

# Import season detector
sys.path.insert(0, '/content/survivorai')
from utils.season_detector import NFLSeasonDetector

detector = NFLSeasonDetector()
current_season, current_week = detector.get_season_info()
status = detector.get_season_status()

print(f"   {status}")
print(f"   Season: {current_season}")
print(f"   Week: {current_week}")

# Set environment variables
os.environ['CURRENT_SEASON'] = str(current_season)
os.environ['CURRENT_WEEK'] = str(current_week)
os.environ['USE_ML_PREDICTIONS'] = 'true'
os.environ['ML_MODEL_TYPE'] = 'ensemble'

# Create .env file
env_content = f"""ODDS_API_KEY={api_key}
CURRENT_SEASON={current_season}
CURRENT_WEEK={current_week}
USE_ML_PREDICTIONS=true
ML_MODEL_TYPE=ensemble
CACHE_DIR=/content/survivorai/cache
ML_MODEL_DIR=/content/survivorai/models
"""

with open('/content/survivorai/.env', 'w') as f:
    f.write(env_content)

print("\n✅ Configuration complete!")
print(f"\n📊 System Settings:")
print(f"   • Season: {current_season}")
print(f"   • Week: {current_week}")
print(f"   • ML Predictions: Enabled")
print(f"   • Model: Ensemble (RF + NN + XGB)")
print(f"   • Real-time data: {'Enabled' if api_key else 'Disabled'}")

# 2️⃣ Real-Time Data Collection

This section:
- Fetches live betting odds from The Odds API
- Scrapes consensus data from SurvivorGrid
- Applies ML predictions
- Blends multiple data sources

**⏱️ Expected time: 10-30 seconds**

In [ ]:
#@title 📡 Fetch Real-Time NFL Data
#@markdown Collects data from multiple sources and applies ML enhancement.

print("📡 Fetching Real-Time NFL Data")
print("=" * 60)

# Import required modules
import sys
sys.path.insert(0, '/content/survivorai')

from data_collection.data_manager import DataManager
import config

# Reload config to pick up environment variables
import importlib
importlib.reload(config)

print(f"\n🎯 Target: Week {config.CURRENT_WEEK} of {config.CURRENT_SEASON} season")
print("\n📊 Data Sources:")

# Initialize data manager
use_odds_api = bool(os.environ.get('ODDS_API_KEY', ''))
use_ml = True

if use_odds_api:
    print("   ✓ The Odds API (real-time betting odds)")
else:
    print("   ⚠ The Odds API (skipped - no API key)")

print("   ✓ SurvivorGrid (consensus picks)")
print("   ✓ ML Models (ensemble predictions)")

print("\n⏳ Fetching data...")

try:
    manager = DataManager(use_odds_api=use_odds_api, use_ml_predictions=use_ml)
    data = manager.get_comprehensive_data(current_week=config.CURRENT_WEEK)
    
    if not data.empty:
        print(f"\n✅ Data collection complete!")
        print(f"\n📈 Data Summary:")
        print(f"   • Total records: {len(data)}")
        print(f"   • Teams: {len(data['team'].unique())}")
        print(f"   • Weeks: {sorted(data['week'].unique())}")
        print(f"   • Data freshness: Real-time")
        
        # Show sample data
        print(f"\n📋 Sample Data (Week {config.CURRENT_WEEK}):")
        week_data = data[data['week'] == config.CURRENT_WEEK].head(5)
        for _, row in week_data.iterrows():
            print(f"   {row['team']:25s} vs {row.get('opponent', 'TBD'):25s} - Win: {row['win_probability']*100:.1f}%")
    else:
        print("⚠️  Warning: No data retrieved. Using fallback mode.")
        data = None
        
except Exception as e:
    print(f"\n❌ Error fetching data: {e}")
    print("   Will attempt to continue with fallback data...")
    data = None

print("\n" + "=" * 60)

# 3️⃣ User Input & Configuration

Enter your pool information and previous picks.

In [ ]:
#@title 👤 Enter Your Pool Information
#@markdown Configure your pool size and previous picks.

from IPython.display import display, HTML

print("👤 Pool Configuration")
print("=" * 60)

# Pool size
pool_size_input = input("\nEnter your pool size (e.g., 50): ")
try:
    pool_size = int(pool_size_input)
    if pool_size < 1:
        pool_size = 50
        print(f"⚠️  Invalid size, using default: {pool_size}")
except:
    pool_size = 50
    print(f"⚠️  Invalid input, using default: {pool_size}")

print(f"✓ Pool size: {pool_size} entries")

# Previous picks
print(f"\n📝 Enter teams you've already used (one per line)")
print("   Type team names (e.g., 'Kansas City Chiefs')")
print("   Press Enter twice when done\n")

used_teams = []
while True:
    team = input(f"Week {len(used_teams) + 1}: ").strip()
    if not team:
        break
    used_teams.append(team)

print(f"\n✅ Configuration complete!")
print(f"\n📊 Your Pool:")
print(f"   • Pool size: {pool_size} entries")
print(f"   • Teams used: {len(used_teams)}")
if used_teams:
    print(f"   • Previous picks:")
    for i, team in enumerate(used_teams, 1):
        print(f"      Week {i}: {team}")
else:
    print(f"   • No teams used yet (first pick!)")

print("\n" + "=" * 60)

# 4️⃣ Optimization & Analysis

This section:
- Runs Hungarian algorithm optimization
- Applies pool-size strategy
- Performs Monte Carlo risk analysis
- Generates recommendations

**⏱️ Expected time: 5-15 seconds**

In [ ]:
#@title 🎯 Run Optimization & Generate Recommendations
#@markdown Calculates optimal picks using advanced algorithms and ML predictions.

print("🎯 Running Optimization")
print("=" * 60)

from optimizer.hungarian_optimizer import SurvivorOptimizer
from optimizer.pool_calculator import PoolCalculator
from analytics.monte_carlo import MonteCarloSimulator

if data is not None and not data.empty:
    print("\n⚙️  Initializing optimizer...")
    optimizer = SurvivorOptimizer(data, used_teams)
    
    print(f"   Available teams: {len(optimizer.available_teams)}")
    print(f"   Weeks to optimize: {config.CURRENT_WEEK}-18")
    
    print("\n🔍 Finding optimal paths...")
    print("   Using Hungarian algorithm (linear sum assignment)")
    print("   Maximizing product of win probabilities...")
    
    top_picks = optimizer.get_top_picks(config.CURRENT_WEEK, n_picks=5)
    
    if top_picks:
        print(f"\n✅ Found {len(top_picks)} optimal paths!")
        
        # Apply pool size adjustments
        print("\n📊 Applying pool-size strategy...")
        pool_calc = PoolCalculator(pool_size)
        strategy = pool_calc.get_strategy_recommendation()
        print(f"   Strategy: {strategy}")
        
        adjusted_picks = pool_calc.adjust_picks_for_pool_size(top_picks)
        
        # Run Monte Carlo on top pick
        print("\n🎲 Running Monte Carlo simulation...")
        print("   Simulating 10,000 season outcomes...")
        
        simulator = MonteCarloSimulator(n_simulations=10000, random_seed=42)
        mc_result = simulator.simulate_path(adjusted_picks[0]['full_path'])
        
        print(f"   ✓ Simulation complete")
        print(f"   95% CI: [{mc_result.confidence_interval_95[0]*100:.2f}%, {mc_result.confidence_interval_95[1]*100:.2f}%]")
        
        print("\n" + "=" * 60)
        print("✅ Analysis complete!")
    else:
        print("\n❌ No valid picks found. Check your configuration.")
        adjusted_picks = []
else:
    print("\n❌ No data available for optimization.")
    print("   Please check your data collection setup.")
    adjusted_picks = []

# 5️⃣ Results & Recommendations

View your optimized picks with complete analysis.

In [ ]:
#@title 📊 Display Recommendations
#@markdown Shows top 5 picks with complete season paths and analysis.

from IPython.display import display, HTML
import pandas as pd

print("\n" + "🏈" * 30)
print(f"\n  TOP {len(adjusted_picks)} RECOMMENDATIONS - WEEK {config.CURRENT_WEEK}")
print("\n" + "🏈" * 30)

if adjusted_picks:
    for i, pick in enumerate(adjusted_picks, 1):
        print("\n" + "=" * 60)
        print(f"\n#{i} PICK: {pick['recommended_team']}")
        print("=" * 60)
        
        # Key metrics
        print(f"\n📈 Key Metrics:")
        print(f"   • This Week Win %:     {pick['win_probability_this_week']*100:.1f}%")
        print(f"   • Win-Out Probability: {pick['overall_win_probability']*100:.2f}%")
        print(f"   • Pool Adjusted Score: {pick.get('composite_score', 0):.3f}")
        print(f"   • Pick Popularity:     {pick['pick_percentage_this_week']*100:.1f}%")
        
        # Complete season path
        print(f"\n📅 Complete Season Path:")
        print(f"\n   {'Week':>4} | {'Team':25s} | {'Opponent':25s} | {'Win %':>7}")
        print("   " + "-" * 75)
        
        cumulative = 1.0
        for p in pick['full_path']:
            week = p['week']
            team = p['team']
            opponent = p.get('opponent', 'TBD')
            win_prob = p['win_probability']
            cumulative *= win_prob
            
            print(f"   {week:4d} | {team:25s} | {opponent:25s} | {win_prob*100:6.1f}%")
        
        print(f"\n   💡 Cumulative Win-Out: {cumulative*100:.2f}%")
        
        # Monte Carlo for top pick
        if i == 1:
            print(f"\n🎲 Monte Carlo Analysis (10,000 simulations):")
            print(f"   • Expected Win-Out: {mc_result.mean_win_out_prob*100:.2f}%")
            print(f"   • 95% Confidence:   [{mc_result.confidence_interval_95[0]*100:.2f}%, {mc_result.confidence_interval_95[1]*100:.2f}%]")
            print(f"   • Standard Dev:     {mc_result.std_win_out_prob*100:.3f}%")
            print(f"\n   📊 Week-by-Week Survival Rates:")
            for idx, rate in enumerate(mc_result.survival_rate_by_week[:5], 1):
                week_num = pick['full_path'][idx-1]['week']
                bar = "█" * int(rate * 20)
                print(f"      Week {week_num:2d}: {bar:20s} {rate*100:5.1f}%")
        
        if i < len(adjusted_picks):
            print("\n")
    
    # Summary
    print("\n" + "=" * 60)
    print("\n🎯 RECOMMENDATION SUMMARY")
    print("=" * 60)
    print(f"\n✅ Analysis complete for {pool_size}-entry pool")
    print(f"📍 Strategy: {strategy}")
    print(f"\n🏆 Top Pick: {adjusted_picks[0]['recommended_team']}")
    print(f"   Win-out probability: {adjusted_picks[0]['overall_win_probability']*100:.2f}%")
    print(f"   Risk level: {'Low' if adjusted_picks[0]['win_probability_this_week'] > 0.7 else 'Medium' if adjusted_picks[0]['win_probability_this_week'] > 0.6 else 'High'}")
    
    print("\n💡 Next Steps:")
    print("   1. Review all 5 options above")
    print("   2. Consider your risk tolerance")
    print("   3. Factor in pool dynamics")
    print("   4. Make your pick with confidence!")
    
    print("\n🏈 Good luck with your survivor pool! 🏈\n")
    
else:
    print("\n❌ No recommendations available.")
    print("   Please check the optimization step for errors.")

# 6️⃣ Export Results (Optional)

Save your results for future reference.

In [ ]:
#@title 💾 Export Results to CSV
#@markdown Save recommendations to a downloadable CSV file.

if adjusted_picks:
    print("💾 Exporting Results")
    print("=" * 60)
    
    # Create summary DataFrame
    summary_data = []
    for i, pick in enumerate(adjusted_picks, 1):
        summary_data.append({
            'Rank': i,
            'Team': pick['recommended_team'],
            'This_Week_Win_Pct': f"{pick['win_probability_this_week']*100:.1f}%",
            'Win_Out_Probability': f"{pick['overall_win_probability']*100:.2f}%",
            'Pool_Score': f"{pick.get('composite_score', 0):.3f}",
            'Pick_Percentage': f"{pick['pick_percentage_this_week']*100:.1f}%",
        })
    
    summary_df = pd.DataFrame(summary_data)
    
    # Save to CSV
    filename = f"/content/survivor_ai_recommendations_week{config.CURRENT_WEEK}.csv"
    summary_df.to_csv(filename, index=False)
    
    print(f"\n✅ Results exported to: {filename}")
    print("\n📥 To download:")
    print("   1. Click the folder icon on the left")
    print(f"   2. Find {filename}")
    print("   3. Right-click and select 'Download'")
    
    # Display summary
    print("\n📊 Summary:")
    display(summary_df)
else:
    print("❌ No results to export.")

---

## 📚 Additional Resources

- **GitHub Repository**: [elliotttmiller/survivorai](https://github.com/elliotttmiller/survivorai)
- **Documentation**: See repository README.md and ARCHITECTURE.md
- **The Odds API**: [the-odds-api.com](https://the-odds-api.com/)

---

## 🔄 Weekly Updates

To use this notebook each week:
1. Run all cells again
2. System auto-detects current week
3. Add your new pick to the list
4. Get fresh recommendations

---

## ⚡ Performance Notes

- **First run**: 3-5 minutes (setup + dependencies)
- **Subsequent runs**: < 1 minute (cached data)
- **Real-time data**: Fetched on each run
- **ML models**: Pre-trained, loaded instantly

---

**Built with cutting-edge research and production-quality engineering** 🏈🏆

*Survivor AI - Maximize your chances of winning your survivor pool*